In [1]:
import sys
sys.path.append('c:\\Users\\name\\Desktop\\deltau')


In [2]:
import pandas as pd
import numpy as np

file_path = "AAPL_data.csv"  
data = pd.read_csv(file_path)

class QuantileStrategy:
    def __init__(self, window=1):
        self.window = window  

    def generate_signals(self, data):
        # Calculate rolling quantiles
        data['25th_Percentile'] = data['Close'].quantile(0.25)
        data['50th_Percentile'] = data['Close'].quantile(0.50)
        data['75th_Percentile'] = data['Close'].quantile(0.75)

        # Generate signals based on quantiles
        signals = pd.Series(0, index=data.index)  # Default to Hold (0)
        signals[data['Close'] < data['25th_Percentile']] = 1  # Buy signal
        signals[data['Close'] > data['75th_Percentile']] = -1 # Sell signal

        # Shift signals to align with decision-making
        signals = signals.shift(1).fillna(0)  # Avoid lookahead bias
        return signals


In [3]:
# Import necessary modules
from deltau.core.backtester import Backtester, Executor

# Instantiate the strategy, executor, and backtester
strategy = QuantileStrategy(window=100)
executor = Executor(initial_capital=600)
backtester = Backtester(strategy=strategy, executor=executor, data=data)

# Run the backtest
portfolio_values, final_metrics = backtester.run()

# Display final metrics
print("Final Metrics:")
for key, value in final_metrics.items():
    print(f"{key}: {value:.2f}")




Buying 1 shares at 74.94999694824219 each for 74.94999694824219 total.
Buying 1 shares at 74.59750366210938 each for 74.59750366210938 total.
Buying 1 shares at 75.79750061035156 each for 75.79750061035156 total.
Buying 1 shares at 77.40750122070312 each for 77.40750122070312 total.
Buying 1 shares at 77.5824966430664 each for 77.5824966430664 total.
Buying 1 shares at 79.23999786376953 each for 79.23999786376953 total.
Buying 1 shares at 78.16999816894531 each for 78.16999816894531 total.
Not enough capital to execute buy order. Capital required: 77.83499908447266, Available capital: 62.2550048828125
Not enough capital to execute buy order. Capital required: 78.80999755859375, Available capital: 62.2550048828125
Not enough capital to execute buy order. Capital required: 79.68250274658203, Available capital: 62.2550048828125
Not enough capital to execute buy order. Capital required: 79.14250183105469, Available capital: 62.2550048828125
Not enough capital to execute buy order. Capital 

In [4]:
from deltau.visualization.tear_sheet import TearSheet


# Create a TearSheet object with the results and metrics
tear_sheet = TearSheet(backtest_results=portfolio_values, metrics=final_metrics)

# Generate the report and show plots
tear_sheet.display_report()

Performance Metrics:
                   Metric       Value
 Initial Portfolio Value  600.000000
   Final Portfolio Value 1607.537525
        Total Return (%)  167.922921
            Sharpe Ratio    0.042563
    Maximum Drawdown (%)  -29.285037
           Sortino Ratio    0.060060
            Win Rate (%)   51.612903
           Profit Factor    1.258543
Average Trade Return (%)    0.169829
          Volatility (%)    1.553881
         Average Win (%)    1.601734
        Average Loss (%)   -1.367130
